In [1]:
%load_ext autoreload
%autoreload 2
from biotorch.benchmark.run import Benchmark

In [2]:
benchmark = Benchmark('/home/albert/projects/biotorch/biotorch/configs/mnist.yaml')

In [3]:
benchmark.run()

Preparing CIFAR10 Dataset and storing data in ./data
Files already downloaded and verified
Files already downloaded and verified
=> Creating model 'resnet18'
Converting ResNet-18 to Feedback Alignment mode
Module has been converted to fa mode:

- All the 20 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully.
- All the 1 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.

Benchmarking model on CIFAR10 Dataset
Linear(in_features=512, out_features=1000, bias=True) tensor(89.9273, device='cuda:0')


/home/albert/projects/biotorch/.venv/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


RuntimeError: hook 'angle_hook' has returned an incorrect number of values (got 1, but expected 5)

In [2]:
from biotorch.models.weight_mirroring.resnet import resnet18, resnet34, resnet50, wide_resnet101_2

In [3]:
model = resnet18()

In [3]:
import biotorch.models as models

In [7]:
model = resnet18(pretrained=False, num_classes=1000)

In [12]:
model = models.__dict__['fa'].__dict__['resnet18']()

Converting ResNet-18 to Feedback Alignment mode
Module has been converted to feedback-alignment mode:

- All the 20 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully.
- All the 1 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.


In [21]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [4]:
import torch
x = torch.randn((10, 3, 224, 224))

In [5]:
model.mirror_weights(x)

> /home/albert/projects/biotorch/biotorch/models/weight_mirroring/resnet.py(354)mirror_weights()
    352 
    353         # Create input noise with the dimensions of the input
--> 354         with torch.no_grad():
    355             input_noise_0 = noise_amplitude * (torch.randn(x.size()))
    356             input_noise = input_noise_0

ipdb> import torch.nn.functional as F
ipdb> n
> /home/albert/projects/biotorch/biotorch/models/weight_mirroring/resnet.py(355)mirror_weights()
    353         # Create input noise with the dimensions of the input
    354         with torch.no_grad():
--> 355             input_noise_0 = noise_amplitude * (torch.randn(x.size()))
    356             input_noise = input_noise_0
    357             input_noise_zero_mean = input_noise - torch.mean(input_noise, dim=0)

ipdb> nn
<module 'torch.nn' from '/home/albert/projects/biotorch/.venv/lib/python3.8/site-packages/torch/nn/__init__.py'>
ipdb> n
> /home/albert/projects/biotorch/biotorch/models/weight_mirror

ipdb>         dB = mirror_learning_rate * F.conv2d(torch.transpose(input_noise, 0, 1), torch.transpose(output_noise, 0, 1), stride=[2, 2], padding=3)
ipdb> dB.size()
torch.Size([3, 64, 60, 60])
ipdb> self.conv1.weight.size()
torch.Size([64, 3, 7, 7])
ipdb> input_noise.size()
torch.Size([10, 3, 224, 224])
ipdb> output_noise.size()
torch.Size([10, 64, 112, 112])
ipdb> exit


BdbQuit: 

In [11]:
import torch
import torch.nn.functional as F
# define the W and B kernels
forward_kernel = torch.randn([16, 16, 3, 3])
backward_kernel = torch.randn([16, 16, 3, 3])
# downsample in the forward pass
noise_input  = torch.randn(1, 16, 12, 12)
noise_output = F.conv2d(noise_input, forward_kernel, stride=[1,1], padding=1)
# upsample in the backward pass
output = F.conv_transpose2d(noise_output, backward_kernel, stride=[1,1], padding=1)
assert(list(output.shape) == list(noise_input.shape))

In [12]:
output.size()

torch.Size([1, 16, 12, 12])

In [13]:
noise_input.size()

torch.Size([1, 16, 12, 12])

In [4]:
benchmark = Benchmark('/home/albert/projects/biotorch/biotorch/configs/mnist.yaml')

In [5]:
benchmark.run()

Preparing CIFAR10 Dataset and storing data in ./data
Files already downloaded and verified
Files already downloaded and verified
=> Creating model 'resnet18'
Converting ResNet-18 to Feedback Alignment mode


TypeError: super(type, obj): obj must be an instance or subtype of type

In [23]:
import torch
import torch.nn.functional as F
x = torch.randn(1, 3, 224, 224)
# define the W and B kernels
W = torch.randn([64, 3, 7, 7])
B = torch.randn([64, 3, 7, 7])
# downsample in the forward pass
y = F.conv2d(x, W, stride=2, padding=3)
# compute the dB update for B
dB = F.conv2d(torch.transpose(x, 0, 1), torch.transpose(y, 0, 1), stride=2, padding=3)

dB = torch.transpose(dB, 0, 1)
print("x t = ", torch.transpose(x, 0, 1).shape)
print("y t = ", torch.transpose(y, 0, 1).shape)
print("x  = ", x.shape)
print("y  = ", y.shape)
print("B  = ", B.shape)
print("dB = ", dB.shape)

x t =  torch.Size([3, 1, 224, 224])
y t =  torch.Size([64, 1, 112, 112])
x  =  torch.Size([1, 3, 224, 224])
y  =  torch.Size([1, 64, 112, 112])
B  =  torch.Size([64, 3, 7, 7])
dB =  torch.Size([64, 3, 60, 60])


In [4]:
m = benchmark.model

In [5]:
def compute_angle_layers(module):
        # Go through all of module nn.module (e.g. network or layer)
        for module_name in module._modules.keys():
            # Get layer
            layer = getattr(module, module_name)
            # Convert layer
            print(layer)
        # Iterate through immediate child modules
        for name, child_module in module.named_children():
            forward_layers(child_module)

In [6]:
forward_layers(m)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
biotorch.layers.fa

<module 'biotorch.layers.fa' from '/home/albert/projects/biotorch/biotorch/layers/fa/__init__.py'>

In [4]:
benchmark.model

BioModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [6]:
from torchvision import models

In [7]:
model = models.resnet18()

In [8]:
import torch

In [10]:
model = torch.nn.DataParallel(model, [1, 2])

In [14]:
torch.save(model.module, 'model.pth')

In [15]:
torch.load('model.pth')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  